## Toy example1

In [1]:
import tensorflow as tf
import numpy as np

# Data
data = "hello"
chars = list(set(data))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }
vocab_size = len(chars)

# Prepare inputs
input_seq = [char_to_ix[ch] for ch in data]
target_seq = [char_to_ix[ch] for ch in data[1:] + data[0]]

input_seq = tf.one_hot(input_seq, vocab_size)  # One-hot encoding
input_seq = tf.expand_dims(input_seq, 0)        # Add batch dimension
target_seq = tf.expand_dims(target_seq, 0)

# Build model
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(10, return_sequences=True),
    tf.keras.layers.Dense(vocab_size)
])

# Compile
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam')

# Train
n_epochs = 1000

for epoch in range(n_epochs):
    with tf.GradientTape() as tape:
        output = model(input_seq)
        loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(
            target_seq, output, from_logits=True))
    
    grads = tape.gradient(loss, model.trainable_variables)
    model.optimizer.apply_gradients(zip(grads, model.trainable_variables))

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.numpy()}')

        # Sampling
        prediction = tf.argmax(output, axis=-1)
        sampled_text = ''.join(ix_to_char[ix.numpy()] for ix in prediction[0])
        print(f'Generated: {sampled_text}')


Epoch 0, Loss: 1.4917757511138916
Generated: heeel
Epoch 100, Loss: 0.6213928461074829
Generated: elloh
Epoch 200, Loss: 0.2752682566642761
Generated: elloh
Epoch 300, Loss: 0.14365044236183167
Generated: elloh
Epoch 400, Loss: 0.08586446195840836
Generated: elloh
Epoch 500, Loss: 0.056807905435562134
Generated: elloh
Epoch 600, Loss: 0.040397919714450836
Generated: elloh
Epoch 700, Loss: 0.030260393396019936
Generated: elloh
Epoch 800, Loss: 0.023553723469376564
Generated: elloh
Epoch 900, Loss: 0.01887591928243637
Generated: elloh


## Toy example 2

In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# Step 1: Download the text
url = 'https://www.gutenberg.org/files/11/11-0.txt'
response = requests.get(url)
text = response.text

print(f"Downloaded text length: {len(text)} characters")



Downloaded text length: 148080 characters
*** START OF THE PROJECT GUTENBERG EBOOK 11 ***

[Illustration]




Alices Adventures in Wond


In [14]:
# Step 2: Preprocess the text
# Only keep ASCII characters to avoid weird symbols
text = ''.join(c for c in text if ord(c) < 128)
print(text[:1000])

# text = "This is GeeksforGeeks a software training institute"


*** START OF THE PROJECT GUTENBERG EBOOK 11 ***

[Illustration]




Alices Adventures in Wonderland

by Lewis Carroll

THE MILLENNIUM FULCRUM EDITION 3.0

Contents

 CHAPTER I.     Down the Rabbit-Hole
 CHAPTER II.    The Pool of Tears
 CHAPTER III.   A Caucus-Race and a Long Tale
 CHAPTER IV.    The Rabbit Sends in a Little Bill
 CHAPTER V.     Advice from a Caterpillar
 CHAPTER VI.    Pig and Pepper
 CHAPTER VII.   A Mad Tea-Party
 CHAPTER VIII.  The Queens Croquet-Ground
 CHAPTER IX.    The Mock Turtles Story
 CHAPTER X.     The Lobster Quadrille
 CHAPTER XI.    Who Stole the Tarts?
 CHAPTER XII.   Alices Evidence




CHAPTER I.
Down the Rabbit-Hole


Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into
the book her sister was reading, but it had no pictures or
conversations in it, and what is the use of a book, thought Alice
without pictures or conversations?


In [16]:
chars = sorted(list(set(text)))
char_to_index = {char: i for i, char in enumerate(chars)}
index_to_char = {i: char for i, char in enumerate(chars)}

seq_length = 3
sequences = []
labels = []

for i in range(len(text) - seq_length):
    seq = text[i:i + seq_length]
    label = text[i + seq_length]
    sequences.append([char_to_index[char] for char in seq])
    labels.append(char_to_index[label])

X = np.array(sequences)
y = np.array(labels)

X_one_hot = tf.one_hot(X, len(chars))
y_one_hot = tf.one_hot(y, len(chars))


In [24]:
EPOCHS = 100
model = Sequential()
model.add(SimpleRNN(50, input_shape=(seq_length, len(chars)), activation='relu'))
model.add(Dense(len(chars), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


import time
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.times = []
    
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_time_start = time.time()
    
    def on_epoch_end(self, epoch, logs=None):
        self.times.append(time.time() - self.epoch_time_start)

# Create callback
time_callback = TimeHistory()

# Train model with callback
history = model.fit(X_one_hot, y_one_hot, epochs=EPOCHS, callbacks=[time_callback])

# model.fit(X_one_hot, y_one_hot, epochs=EPOCHS)


Epoch 1/100
4526/4526 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - accuracy: 0.3130 - loss: 2.6076
Epoch 2/100
4526/4526 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.4361 - loss: 2.0186
Epoch 3/100
4526/4526 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.4570 - loss: 1.9229
Epoch 4/100
4526/4526 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.4667 - loss: 1.8753
Epoch 5/100
4526/4526 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.4769 - loss: 1.8392
Epoch 6/100
4526/4526 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.4840 - loss: 1.8126
Epoch 7/100
4526/4526 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.4918 - loss: 1.7854
Epoch 8/100
4526/4526 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.4913 - loss: 1.7734
Epoch 9/100
4526/4526 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.4963 - loss: 1.7556
Epoch 10/100
4526/4526 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.5001 - loss: 1.7417
Epoch 11/100
4526/4526 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.5009 - loss: 1.7361
Epoch 12

In [25]:
# After training, you can see time per epoch
print("\nTime per epoch (in seconds):", time_callback.times)

# Total training time in minutes
total_time_min = sum(time_callback.times) / 60
print(f"Total training time: {total_time_min:.2f} minutes")



Time per epoch (in seconds): [12.50995397567749, 11.113578081130981, 10.49731159210205, 10.757098197937012, 10.760200262069702, 10.203116178512573, 10.135388374328613, 10.34597659111023, 11.045531749725342, 20.607378005981445, 10.784028053283691, 11.100220918655396, 11.408073425292969, 11.050654888153076, 11.265387058258057, 11.448534965515137, 11.0776207447052, 20.610201358795166, 11.087640047073364, 20.63566017150879, 11.101003885269165, 20.61864686012268, 20.759530544281006, 20.504194736480713, 20.5325186252594, 20.66722846031189, 11.168475866317749, 20.42527484893799, 10.750025510787964, 10.51625394821167, 11.300658464431763, 19.580969095230103, 10.353837251663208, 10.241546630859375, 11.282840728759766, 20.78786277770996, 20.44586491584778, 11.318994283676147, 11.292000770568848, 11.344514846801758, 11.352830410003662, 11.358193635940552, 20.80751323699951, 20.490063190460205, 11.362921714782715, 20.65556240081787, 11.4129958152771, 20.71203875541687, 20.423572778701782, 20.76060

In [26]:
start_seq = "Alice"
generated_text = start_seq

for i in range(50):
    x = np.array([[char_to_index[char] for char in generated_text[-seq_length:]]])
    x_one_hot = tf.one_hot(x, len(chars))
    prediction = model.predict(x_one_hot)
    next_index = np.argmax(prediction)
    next_char = index_to_char[next_index]
    generated_text += next_char

print("Generated Text:")
print(generated_text)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━